In [ ]:
!pip install -U neo4j
!pip install -U pandas
!pip install 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 6.0 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116069 sha256=0b5759c8e21711e13d9d7787049cf2c9ff69cb1506397124977b59bc3b3a3a19
  Stored in directory: /root/.cache/pip/wheels/db/dd/76/acacd519878f133f2f869aec70db548d89e04013209c3c62bc
Successfully built neo4j


In [16]:
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import itertools
import os
from time import sleep
import re
from pandas.core.frame import DataFrame

In [32]:
class Gena:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()
    
    def create_nodes(self, node_df):
        with self.driver.session() as session:
            create_nodes_query = "CREATE "
            nodes = []
            for idx, row in node_df.iterrows():
                unique_id = row['ID']
                name = row['Preferred label'].replace('"', '')
                label = row['Label'].upper()
                synonyms = row['Synonyms'].replace('"', '')
                nodes.append("(n" + str(idx) + ":" + label + " {id: " + f'"{unique_id}"' + ", name: " + f'"{name}"' + ", synonyms:" + f'"{synonyms}"' + "})")
        
        with open("test_nodes.txt", "w", encoding="utf-8") as f:
            print("write query to file.")
            f.write(create_nodes_query + ",".join(nodes))
        session.run(create_nodes_query + ",".join(nodes))
                
    def create_relationships(self, relation_df):
        with self.driver.session() as session:
            for idx, row in relation_df.iterrows():
                id_1 = row['ID_1']
                label_1 = row['Type_E1'].upper()
                id_2 = row['ID_2']
                label_2 = row['Type_E2'].upper()
                relation = row['Relation'].upper().replace(' ', '_').replace('"', '')

                had_relation = [result for result in session.run("MATCH p = (node_1)-[r:`CID`]->(node_2) WHERE (node_1.id = $id_1 AND node_2.id = $id_2) RETURN nodes(p)", id_1=id_1, id_2=id_2)]
                if len(had_relation) > 0:
                    try:
                        pmid_source = str(int(row['PMID']))
                        create_relation = f'MATCH (node_1)-[r:CID]->(node_2)\
                        \nWHERE (node_1.id = "{id_1}" AND node_2.id = "{id_2}")\
                        \nSET r.detail = (CASE r.detail WHEN null THEN ["{relation}"] ELSE r.detail + ["{relation}"] END), r.source = r.source + ["{pmid_source}"]'
                        session.run(create_relation)
                    except Exception as e:
                        print(idx)
                        print(e)
                        print('---')
                else:
                    create_relation = f'MATCH (node_1), (node_2) WHERE (node_1.id = "{id_1}" AND node_2.id = "{id_2}") MERGE ((node_1)-[r:`{relation}`' + ' {source:["' + row["PMID"] + '"]}]->(node_2))'
                    session.run(create_relation)

    def delete_data(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")

In [30]:
## Call GENA
uri = "neo4j+s://8451226e.databases.neo4j.io"
user = "neo4j"
password = "RWY0rgJWWysmEbk808JCUv74jF-VrKBEfchJxy1vp5Q"
gena = Gena(uri, user, password)

In [12]:
import time
RESULT_PATH = "../DATASET/results/final_data"
nodes_df = pd.read_csv(f"{RESULT_PATH}/ID_to_ENTITIES_final.csv")
pre_count = 0
count = 2000
while True:
    print(f"Create nodes from {pre_count} to {count}")
    gena.create_nodes(nodes_df[pre_count:count])
    if count > len(nodes_df):
        break
    pre_count = count
    count += 2000
    time.sleep(2)

Create nodes from 0 to 2000
write query to file.
Create nodes from 2000 to 4000
write query to file.
Create nodes from 4000 to 6000
write query to file.
Create nodes from 6000 to 8000
write query to file.
Create nodes from 8000 to 10000
write query to file.
Create nodes from 10000 to 12000
write query to file.
Create nodes from 12000 to 14000
write query to file.
Create nodes from 14000 to 16000
write query to file.
Create nodes from 16000 to 18000
write query to file.
Create nodes from 18000 to 20000
write query to file.
Create nodes from 20000 to 22000
write query to file.
Create nodes from 22000 to 24000
write query to file.
Create nodes from 24000 to 26000
write query to file.
Create nodes from 26000 to 28000
write query to file.
Create nodes from 28000 to 30000
write query to file.


In [27]:
## Create Relationship
relationships = pd.read_csv(f"{RESULT_PATH}/gena_data_final.csv")
pre_count = 0
count = 4000
reverse_relation = relationships[::-1].reset_index(drop=True)
while True:
    print(f"Create relationship from {pre_count} to {count}")
    gena.create_relationships(reverse_relation[pre_count:count])
    if count > len(reverse_relation):
        break
    pre_count = count
    count += 4000
    sleep(3)

Create relationship from 0 to 4000
Create relationship from 4000 to 8000
Create relationship from 8000 to 12000
10215
{code: Neo.ClientError.Statement.TypeError} {message: Neo4j only supports a subset of Cypher types for storage as singleton or array properties. Please refer to section cypher/syntax/values of the manual for more details.}
---
10407
{code: Neo.ClientError.Statement.TypeError} {message: Neo4j only supports a subset of Cypher types for storage as singleton or array properties. Please refer to section cypher/syntax/values of the manual for more details.}
---
Create relationship from 12000 to 16000
Create relationship from 16000 to 20000
17413
{code: Neo.ClientError.Statement.TypeError} {message: Neo4j only supports a subset of Cypher types for storage as singleton or array properties. Please refer to section cypher/syntax/values of the manual for more details.}
---
Create relationship from 20000 to 24000
Create relationship from 24000 to 28000
24874
{code: Neo.ClientError.

In [34]:
cid_relation = reverse_relation.iloc[[10215, 10417, 17413, 24874, 28331, 29734, 30433, 30749, 30981, 31776, 32015, 33512, 33718, 35366, 35808,
                      38663, 38967, 40255, 41356, 41922, 42688], :]
gena.create_relationships(cid_relation)